In [9]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime
import time
import numpy as np

load_dotenv()

True

In [3]:
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY not found. Make sure it's set in your .env file.")
else:
    print(API_KEY)

jahhk5gZWG3GS7AH0FA_emtKvSZbfCWK


In [4]:
def fetch_stock_data(ticker, start_date, end_date, api_key):
    URL = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/hour/{start_date}/{end_date}"
    params = {
        "apiKey": api_key,  # ✅ Correct parameter name
        "adjusted": "true",
        "sort": "asc",       # ✅ Keeps results in ascending (oldest → newest) order
        "limit": 50000
    }

    print(f"Fetching data for {ticker} from {start_date} to {end_date}...")
    response = requests.get(URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and data['results']:
            df = pd.DataFrame(data['results'])
            df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 't': 'timestamp', 'v': 'volume'})
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df['ticker'] = ticker
            return df
        else:
            print(f"No results found for {ticker}.")
            return pd.DataFrame()
    else:
        print(f"Error fetching data for {ticker}: {response.status_code} - {response.text}")
        return pd.DataFrame()


In [5]:
tickers_to_fetch = ['XOM', 'CVX', 'WTI', 'COP', 'BP', 'EOG', 'PBR']
start_date_str = "2020-01-01"
end_date_str = datetime.today().strftime('%Y-%m-%d')

# 2. Loop through tickers
all_data = []
for i, ticker in enumerate(tickers_to_fetch, start=1):
    stock_df = fetch_stock_data(ticker, start_date_str, end_date_str, API_KEY)
    if not stock_df.empty:
        all_data.append(stock_df)
    if i%5 == 0 and i < len(tickers_to_fetch):
        print(f"\n⏳ Processed {i} tickers, waiting 60 seconds to avoid rate limits...\n")
        time.sleep(60)

# 3. Combine and save
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    output_path = "../data/raw/oil_stocks_hourly_raw.csv"
    final_df.to_csv(output_path, index=False)
    print(f"\nSuccessfully fetched and saved data for {len(all_data)} tickers.")
    print(f"Raw data saved to: {output_path}")
    print(final_df.head())
else:
    print("No data was fetched. Please check your tickers, date range, and API key.")

Fetching data for XOM from 2020-01-01 to 2025-10-13...
Fetching data for CVX from 2020-01-01 to 2025-10-13...
Fetching data for WTI from 2020-01-01 to 2025-10-13...
Fetching data for COP from 2020-01-01 to 2025-10-13...
Fetching data for BP from 2020-01-01 to 2025-10-13...

⏳ Processed 5 tickers, waiting 60 seconds to avoid rate limits...

Fetching data for EOG from 2020-01-01 to 2025-10-13...
Fetching data for PBR from 2020-01-01 to 2025-10-13...

Successfully fetched and saved data for 7 tickers.
Raw data saved to: ../data/raw/oil_stocks_hourly_raw.csv
     volume        vw    open   close    high     low           timestamp  \
0    6556.0  107.3982  107.16  107.41  107.55  107.16 2023-10-13 08:00:00   
1   15550.0  107.7053  107.49  107.68  108.00  107.40 2023-10-13 09:00:00   
2   36300.0  108.0503  107.84  107.80  108.29  107.79 2023-10-13 10:00:00   
3   68418.0  107.6703  107.78  107.82  107.85  107.50 2023-10-13 11:00:00   
4  213367.0  107.8121  107.81  107.95  108.14  106.47 

In [6]:
raw_data_path = "../data/raw/oil_stocks_hourly_raw.csv"
df = pd.read_csv(raw_data_path)

In [7]:
print("Successfully loaded raw data with the following details:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(df.head())

Successfully loaded raw data with the following details:
Shape: (10514, 9)
Columns: ['volume', 'vw', 'open', 'close', 'high', 'low', 'timestamp', 'n', 'ticker']
     volume        vw    open   close    high     low            timestamp  \
0    6556.0  107.3982  107.16  107.41  107.55  107.16  2023-10-13 08:00:00   
1   15550.0  107.7053  107.49  107.68  108.00  107.40  2023-10-13 09:00:00   
2   36300.0  108.0503  107.84  107.80  108.29  107.79  2023-10-13 10:00:00   
3   68418.0  107.6703  107.78  107.82  107.85  107.50  2023-10-13 11:00:00   
4  213367.0  107.8121  107.81  107.95  108.14  106.47  2023-10-13 12:00:00   

      n ticker  
0    60    XOM  
1   221    XOM  
2   380    XOM  
3   803    XOM  
4  1632    XOM  


In [10]:
# --- 2. Calculate the Log Percent Change ---
# This is the primary feature the model will use.
# It's calculated using the 'open' and 'close' columns you provided.
# We add a tiny number (epsilon) to the denominator to prevent division by zero errors.
epsilon = 1e-8
df['log_percent_change'] = np.log(df['close'] / (df['open'] + epsilon))

In [11]:
# --- 3. Restructure (Pivot) the DataFrame ---
# The model needs the data in a "wide" format where each ticker is a feature column.
# We'll set 'timestamp' as the row index and create a new column for each unique 'ticker'.
processed_df = df.pivot_table(index='timestamp', columns='ticker', values='log_percent_change')

# After pivoting, missing values can appear. We'll fill them using the last valid observation.
processed_df.fillna(method='ffill', inplace=True)
processed_df.fillna(method='bfill', inplace=True) # Fill any remaining NaNs at the start

print("\nData has been pivoted. The new format is:")
print(f"Shape: {processed_df.shape}")
print(processed_df.head())


Data has been pivoted. The new format is:
Shape: (2002, 7)
ticker                     BP           COP       CVX           EOG       PBR  \
timestamp                                                                       
2023-10-13 08:00:00  0.004252 -8.164591e-11  0.000737 -7.642342e-11  0.008606   
2023-10-13 09:00:00 -0.002498 -8.141332e-11  0.002206 -7.642342e-11  0.004613   
2023-10-13 10:00:00  0.002746 -8.141332e-11 -0.001282 -7.642342e-11  0.001970   
2023-10-13 11:00:00 -0.001248  2.354757e-03  0.001346  7.621653e-05  0.009859   
2023-10-13 12:00:00  0.000962  8.178514e-03  0.003599 -4.561011e-03  0.007187   

ticker                        WTI       XOM  
timestamp                                    
2023-10-13 08:00:00  2.472187e-03  0.002330  
2023-10-13 09:00:00  2.472187e-03  0.001766  
2023-10-13 10:00:00 -2.481390e-09 -0.000371  
2023-10-13 11:00:00  7.380105e-03  0.000371  
2023-10-13 12:00:00  2.472187e-03  0.001298  


C:\Users\shara\AppData\Local\Temp\ipykernel_19300\2683735512.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  processed_df.fillna(method='ffill', inplace=True)
C:\Users\shara\AppData\Local\Temp\ipykernel_19300\2683735512.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  processed_df.fillna(method='bfill', inplace=True) # Fill any remaining NaNs at the start


In [12]:
# --- 4. Save the Final Processed File ---
# This clean file will be used by your PyTorch Dataset for model training.
processed_data_path = "../data/processed/oil_stocks_hourly_processed.csv"
processed_df.to_csv(processed_data_path)

print(f"\nPreprocessing complete. Model-ready data saved to: {processed_data_path}")


Preprocessing complete. Model-ready data saved to: ../data/processed/oil_stocks_hourly_processed.csv
